<a href="https://colab.research.google.com/github/sevaraakhbaeva/twitter-sentiment-analysis-ML23/blob/danik/Twitter_final_11_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 - Installing dependencies


In [1]:
!pip3 install tueplots==0.0.5
!pip3 install sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=11316f7a69736ba3452888035b5c43309ef0b14791b14d324d6a06fdb9f92348
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
# importing dependencies
import os

import pandas as pd
pd.set_option('display.max_rows', 100)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tueplots import bundles
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['text.usetex'] = True

from wordcloud import WordCloud

plt.rcParams.update(bundles.icml2022())
import tueplots.constants.color.palettes as tue_palettes

from sentence_transformers import SentenceTransformer

from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/ML_TwitterCh'

import xgboost as xgb
import sklearn.metrics as skm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ML_TwitterCh


## 2 - Pre-processing

In [6]:
# importing datasets for training and tesing the models
team_id = '30' #put your team id here
split = 'test_1' # replace by 'test_2' for FINAL submission

df = pd.read_csv('tweets_train.csv')
df_test = pd.read_csv(f'tweets_{split}.csv')

In [7]:
df.head()
# there is no reason to use the column called possibly_sensitive because all the values are equal to 0s

,id,text,type,author_id,possibly_sensitive,retweet_count,quote_count,reply_count,like_count,followers_count,following_count,tweet_count,listed_count,words,score_compound,sentiment
0,1532325760148590593,@xbresson British Alps :-),replied_to,62044012,False,0,0,1,2,33022,3383,5246,359,"['british', 'alps']",0.0000,neutral
1,1312410689059913731,RT @Aistats2020: Videos presentations of paper...,retweeted,1290762290224984064,False,52,0,0,0,17570,541,703,222,"['videos', 'presentations', 'papers', 'keynote...",0.0000,neutral
2,1564467011781926913,I hope I would be able to talk more about this...,quoted,3363584909,False,1,0,0,3,65506,113,15406,856,"['hope', 'would', 'able', 'talk', 'balcony', '...",0.4862,neutral
3,1473395733344788481,RT @dlbcnai: Keynote by Joan Bruna (@joanbruna...,retweeted,1071640880,False,35,0,0,0,21847,555,1346,264,"['keynote', 'joan', 'bruna', 'geometric', 'dee...",0.3612,neutral
4,1611495430843502593,@annargrs @Michael_J_Black @AllenHW0 @CSProfKG...,replied_to,48008938,False,0,0,1,5,427851,582,12949,6488,"['process', 'science', 'relies', 'much', 'basi...",-0.3182,neutral


In [8]:
df.isnull().sum().sort_values(ascending=False)/df.shape[0]

id                    0.0
text                  0.0
type                  0.0
author_id             0.0
possibly_sensitive    0.0
retweet_count         0.0
quote_count           0.0
reply_count           0.0
like_count            0.0
followers_count       0.0
following_count       0.0
tweet_count           0.0
listed_count          0.0
words                 0.0
score_compound        0.0
sentiment             0.0
dtype: float64

In [9]:
# last pre-processing step
df['words_str'] = df['words'].apply(lambda words: ' '.join(eval(words)))
df_test['words_str'] = df_test['words'].apply(lambda words: ' '.join(eval(words)))

In [10]:
# checking distribution of sentiment
num_total = len(df)
num_pos = len(df[df.sentiment == 'positive'])
num_neu = len(df[df.sentiment == 'neutral'])
num_neg = len(df[df.sentiment == 'negative'])

print(f"Num. positive tweets: {num_pos} ({num_pos/num_total*100:.2f}%)")
print(f"Num. negative tweets: {num_neg} ({num_neg/num_total*100:.2f}%)")
print(f"Num. neutral tweets: {num_neu} ({num_neu/num_total*100:.2f}%)")

Num. positive tweets: 2228 (27.85%)
Num. negative tweets: 482 (6.02%)
Num. neutral tweets: 5290 (66.12%)


In [11]:
longest_sentence = df['words_str'].apply(lambda x: len(x.split())).max()
print("Length of the Longest Sentence:", longest_sentence)

Length of the Longest Sentence: 38


In [12]:
# getting text-embeddings
name = "paraphrase-xlm-r-multilingual-v1"
# name = "nli-bert-base"
model = SentenceTransformer(name)

In [13]:
sentences = list(df.words_str.values)
sentence_embeddings = model.encode(sentences)

In [14]:
X = sentence_embeddings
y = df.score_compound.values

"""
from sklearn.preprocessing import MinMaxScaler
selected_columns = ['followers_count', 'listed_count']

# Extract the selected columns from the DataFrame and convert them to a NumPy array
additional_features = df[selected_columns].values

# additional_features = np.array(X_other)
scaler = MinMaxScaler()
scaled_additional_features = scaler.fit_transform(additional_features)
X = np.concatenate((X, scaled_additional_features), axis=1)
"""
print(f"X: {X.shape}")
print(f"y: {y.shape}")

X: (8000, 768)
y: (8000,)


In [15]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# perform PCA for dimensionality reduction
n_components = 350
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X)

# Fitting linear regression

In [16]:
# splitting data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

datasets = [ [X_train, y_train], [X_val, y_val] ]

In [ ]:
# normal linear regression model
reg = linear_model.LinearRegression(fit_intercept=True, copy_X=True, n_jobs=None, positive=False)
reg = reg.fit(X_train, y_train)

# evaluating our predictor quantitatively
for split_name, dataset in zip(['train', 'valididation'], datasets):
    X_i, y_i = dataset
    y_pred = reg.predict(X_i)
    rmse = np.sqrt(skm.mean_squared_error(y_i, y_pred))
    print(f'\nSplit: {split_name}')
    print(f"\tRMSE: {rmse:.8f}")
    mae = skm.mean_absolute_error(y_i, y_pred)
    print(f"\tMAE: {mae:.8f}")


Split: train
	RMSE: 0.29168134
	MAE: 0.22592962

Split: valididation
	RMSE: 0.32186828
	MAE: 0.24913720


# Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNet

grid_enet={'alpha' : [0.05,0.005, 0.0005, 0.00005],
          'l1_ratio': np.linspace(0.1,0.9, 8)}
enet=ElasticNet()
enet_cv=GridSearchCV(enet, grid_enet, cv=5)
enet = enet_cv.fit(X_train,y_train)
print("best hpyerparameters :",enet_cv.best_params_)
print("Score :",enet_cv.best_score_)

In [ ]:
# evaluating our predictor quantitatively
for split_name, dataset in zip(['train', 'valididation'], datasets):
    X_i, y_i = dataset
    y_pred = enet.predict(X_i)
    rmse = np.sqrt(skm.mean_squared_error(y_i, y_pred))
    print(f'\nSplit: {split_name}')
    print(f"\tRMSE: {rmse:.8f}")
    mae = skm.mean_absolute_error(y_i, y_pred)
    print(f"\tMAE: {mae:.8f}")

# FFNN

In [ ]:
import tensorflow as tf

# Build a simple feedforward neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(770,)),  # Assuming your sentence embeddings have size 768
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Regression task, so no activation
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

# evaluating our predictor quantitatively
for split_name, dataset in zip(['train', 'valididation'], datasets):
    X_i, y_i = dataset
    y_pred = model.predict(X_i)
    rmse = np.sqrt(skm.mean_squared_error(y_i, y_pred))
    print(f'\nSplit: {split_name}')
    print(f"\tRMSE: {rmse:.8f}")
    mae = skm.mean_absolute_error(y_i, y_pred)
    print(f"\tMAE: {mae:.8f}")

Epoch 1/10
90/90 [==============================] - 7s 10ms/step - loss: 0.1350 - val_loss: 0.1139
Epoch 2/10
90/90 [==============================] - 1s 6ms/step - loss: 0.1043 - val_loss: 0.1113
Epoch 3/10
90/90 [==============================] - 1s 6ms/step - loss: 0.0956 - val_loss: 0.1182
Epoch 4/10
90/90 [==============================] - 0s 4ms/step - loss: 0.0877 - val_loss: 0.1130
Epoch 5/10
90/90 [==============================] - 0s 3ms/step - loss: 0.0833 - val_loss: 0.1160
Epoch 6/10
90/90 [==============================] - 0s 3ms/step - loss: 0.0740 - val_loss: 0.1156
Epoch 7/10
90/90 [==============================] - 0s 3ms/step - loss: 0.0676 - val_loss: 0.1181
Epoch 8/10
90/90 [==============================] - 0s 3ms/step - loss: 0.0618 - val_loss: 0.1226
Epoch 9/10
90/90 [==============================] - 0s 3ms/step - loss: 0.0541 - val_loss: 0.1235
Epoch 10/10
200/200 [==============================] - 0s 2ms/step

Split: train
	RMSE: 0.21424667
	MAE: 0.15610332
5

# Fitting lasso regression
Best performing model so far (with alpha = 0.001)



In [ ]:
lasso = Lasso()
reg_lasso = Lasso(0.0001)
reg_lasso.fit(X_train, y_train)

Lasso(alpha=0.0001)

In [ ]:
# Evaluate our predictor quantitatively
for split_name, dataset in zip(['train', 'valididation'], datasets):
    X_i, y_i = dataset
    y_pred = reg_lasso.predict(X_i)
    rmse = np.sqrt(skm.mean_squared_error(y_i, y_pred))
    print(f'\nSplit: {split_name}')
    print(f"\tRMSE: {rmse:.8f}")
    mae = skm.mean_absolute_error(y_i, y_pred)
    print(f"\tMAE: {mae:.8f}")
    r2 = r2_score(y_i, y_pred)
    print('R^2 score on validation set:', r2)


Split: train
	RMSE: 0.29632375
	MAE: 0.22889297
R^2 score on validation set: 0.5782751608381844

Split: valididation
	RMSE: 0.31571659
	MAE: 0.24658687
R^2 score on validation set: 0.49199910986665596


#Support Vector Machine

In [ ]:
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svr_model.fit(X_train, y_train)

SVR()

In [ ]:
# Evaluate our predictor quantitatively
for split_name, dataset in zip(['train', 'valididation'], datasets):
    X_i, y_i = dataset
    y_pred = svr_model.predict(X_i)
    rmse = np.sqrt(skm.mean_squared_error(y_i, y_pred))
    print(f'\nSplit: {split_name}')
    print(f"\tRMSE: {rmse:.8f}")
    mae = skm.mean_absolute_error(y_i, y_pred)
    print(f"\tMAE: {mae:.8f}")
    r2 = r2_score(y_i, y_pred)
    print('R^2 score on validation set:', r2)


Split: train
	RMSE: 0.12657183
	MAE: 0.10119923
R^2 score on validation set: 0.9230567789467428

Split: valididation
	RMSE: 0.30964874
	MAE: 0.23850676
R^2 score on validation set: 0.5113383101646936


# Linear Classification - log reg et al

In [ ]:
from imblearn.over_sampling import SMOTE

X = sentence_embeddings
y_text = df.sentiment.values
le = preprocessing.LabelEncoder()
le.fit(y_text)
print(f'Original classes {le.classes_}')
print(f'Corresponding numeric classes {le.transform(le.classes_)}')
y =le.transform(y_text)
print(f"X: {X.shape}")
print(f"y: {y.shape} {np.unique(y)}")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform SMOTE oversampling on the training set
oversampler = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# Print the class distribution after oversampling
unique_classes, class_counts = np.unique(y_train_resampled, return_counts=True)
print("Class distribution after oversampling:")
for cls, count in zip(unique_classes, class_counts):
    print(f"Class {cls}: {count}")



Original classes ['negative' 'neutral' 'positive']
Corresponding numeric classes [0 1 2]
X: (8000, 768)
y: (8000,) [0 1 2]
Class distribution after oversampling:
Class 0: 4200
Class 1: 4200
Class 2: 4200


In [ ]:
# Encode the target variable using LabelEncoder
le = preprocessing.LabelEncoder()
y_train_resampled = le.fit_transform(y_train_resampled)
y_val_encoded = le.transform(y_val)


In [ ]:
datasets = [
    [X_train_resampled, y_train_resampled],
    [X_val, y_val_encoded]
]
clf = linear_model.LogisticRegression(penalty=None,
                                      dual=False,
                                      tol=0.0001,
                                      C=1.0,
                                      fit_intercept=True,
                                      intercept_scaling=1,
                                      class_weight='balanced', # None, balanced
                                      random_state=None,
                                      solver='lbfgs',
                                      max_iter=1000,
                                      multi_class='auto',
                                      verbose=0,
                                      warm_start=False,
                                      n_jobs=None,
                                      l1_ratio=None
                                     )

In [ ]:
clf = clf.fit(X_train_resampled, y_train_resampled)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
for split_name, dataset in zip(['train', 'validation'], datasets):
    X_i, y_i = dataset
    y_pred = clf.predict(X_i)
    print(f'\nSplit: {split_name}')
    print(skm.classification_report(y_i, y_pred))


Split: train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4200
           1       0.90      0.89      0.89      4200
           2       0.89      0.90      0.90      4200

    accuracy                           0.93     12600
   macro avg       0.93      0.93      0.93     12600
weighted avg       0.93      0.93      0.93     12600


Split: validation
              precision    recall  f1-score   support

           0       0.30      0.44      0.36        87
           1       0.85      0.79      0.82      1090
           2       0.61      0.66      0.63       423

    accuracy                           0.74      1600
   macro avg       0.59      0.63      0.60      1600
weighted avg       0.76      0.74      0.75      1600



In [ ]:
accuracy = clf.score(X_val, y_val_encoded)
print("Logistic Regression Validation accuracy:", accuracy)

Logistic Regression Validation accuracy: 0.738125


In [ ]:
# Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_resampled, y_train_resampled)
rf_accuracy = rf_clf.score(X_val, y_val_encoded)
print("Random Forest Validation Accuracy:", rf_accuracy)

Random Forest Validation Accuracy: 0.755625


In [ ]:
# Gradient Boosting classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_clf.fit(X_train_resampled, y_train_resampled)
gb_accuracy = gb_clf.score(X_val, y_val_encoded)
print("Gradient Boosting Validation Accuracy:", gb_accuracy)

Gradient Boosting Validation Accuracy: 0.73125


In [ ]:
# SVC classifier
svm_clf = SVC(kernel='rbf', random_state=42)
svm_clf.fit(X_train_resampled, y_train_resampled)
svm_accuracy = svm_clf.score(X_val, y_val_encoded)
print("SVM Validation Accuracy:", svm_accuracy)

for split_name, dataset in zip(['train', 'validation'], datasets):
    X_i, y_i = dataset
    y_pred = svm_clf.predict(X_i)
    print(f'\nSplit: {split_name}')
    print(skm.classification_report(y_i, y_pred))

SVM Validation Accuracy: 0.796875

Split: train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4200
           1       0.97      0.97      0.97      4200
           2       0.98      0.97      0.97      4200

    accuracy                           0.98     12600
   macro avg       0.98      0.98      0.98     12600
weighted avg       0.98      0.98      0.98     12600


Split: validation
              precision    recall  f1-score   support

           0       0.45      0.34      0.39        87
           1       0.83      0.90      0.86      1090
           2       0.75      0.63      0.68       423

    accuracy                           0.80      1600
   macro avg       0.68      0.62      0.65      1600
weighted avg       0.79      0.80      0.79      1600



In [ ]:
# KNN classifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_resampled, y_train_resampled)
knn_accuracy = knn_clf.score(X_val, y_val_encoded)
print("KNN Validation Accuracy:", knn_accuracy)

KNN Validation Accuracy: 0.31625


In [ ]:
# Naive Bayes classifier
nb_clf = GaussianNB()
nb_clf.fit(X_train_resampled, y_train_resampled)
nb_accuracy = nb_clf.score(X_val, y_val_encoded)
print("Naive Bayes Validation Accuracy:", nb_accuracy)

Naive Bayes Validation Accuracy: 0.7375


In [ ]:
# Decision Tree classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train_resampled, y_train_resampled)
dt_accuracy = dt_clf.score(X_val, y_val_encoded)
print("Decision Tree Validation Accuracy:", dt_accuracy)

Decision Tree Validation Accuracy: 0.563125


In [ ]:
# XGBoost classifier
xgb_clf = xgb.XGBClassifier(random_state=42)
xgb_clf.fit(X_train_resampled, y_train_resampled)
xgb_accuracy = xgb_clf.score(X_val, y_val_encoded)
print("XGBoost Validation Accuracy:", xgb_accuracy)

XGBoost Validation Accuracy: 0.76625


# Llama

In [ ]:
from transformers import LlamaModel, LlamaConfig

# Initializing a LLaMA llama-7b style configuration
configuration = LlamaConfig()

# Initializing a model from the llama-7b style configuration
model = LlamaModel(configuration)

# Accessing the model configuration
configuration = model.config